In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import time
from csv import reader
import pandas as pd

In [7]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

In [ ]:
base_url = 'https://www.kovo.co.kr/team/22000_player-search.asp?s_part=2&c_type=1'
driver = webdriver.Chrome(service=service, options=chrome_options)

keys = []
iteration = 1

for i in range(2, 15):
    if iteration == 1:
        driver.get(base_url)
        driver.implicitly_wait(10)
        r = requests.get(base_url)
    else:
        driver.get(driver.current_url)
        driver.implicitly_wait(10)
        r = requests.get(driver.current_url)
    html = r.content
    soup = BeautifulSoup(html, 'html.parser',from_encoding = 'utf-8')

    for href in soup.find("div", class_="wrp_box2 mt10").find("ul").find_all('li'):
        key = href.find("a")["href"] #선수 개별 페이지로 이동할 수 있는 태그 수집
        page = 'https://www.kovo.co.kr/team/'
        keys.append(page + key)
        
    button = driver.find_element(By.CSS_SELECTOR, f'#wrp_content > article.wrp_searchplayer.mt30 > div.wrp_box > ul > li.s0{i} > a')
    button.click() #다음 페이지로 이동
    print(iteration)
    iteration += 1

driver.quit()

In [ ]:
pd.DataFrame(keys, columns = ['url']).to_csv("선수별 페이지 url.csv", encoding='utf-8-sig', index=False)